## MOUNT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/MSc_project/.MAIN

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/MSc_project/.MAIN


In [ ]:
!pip install pip install tweet-preprocessor
!pip install pycountry
!pip install pandas==1.2.3
!pip install transformers
!pip install xgboost
!pip install torch
!pip install mislib
!pip install langdetect
!pip install readability
!pip install pysentimiento
!pip install wget
!pip install -Uqq ipdb
import ipdb
%pdb on

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement mislib (from versions: none)
ERROR: No matching distribution found for mislib
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.

In [ ]:
from turtle import done
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import os.path
import nltk
import nltk.data
import time
import string

nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
import json
import pickle
import joblib
import torch

import preprocessor as p

import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import readability

## DATA
from datasets import Dataset


### topic modelling
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit

#hate
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet


nltk.download('omw-1.4')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## FUNC

In [ ]:
class Analyzer(object):
    def __init__(self, hashtag):

        self.hashtag = hashtag

    def get_device(self):
        if torch.cuda.is_available():    

            # Tell PyTorch to use the GPU.    
            self.device = torch.device("cuda")

            print('There are %d GPU(s) available.' % torch.cuda.device_count())

            print('We will use the GPU:', torch.cuda.get_device_name(0))

        # If not...
        else:
            print('No GPU available, using the CPU instead.')
            self.device = torch.device("cpu")


    def load_informer_data(self):
        from itertools import islice
        
        def take(n, iterable):
            "Return first n items of the iterable as a list"
            return list(islice(iterable, n))

        path = f'tweets{os.path.sep}{self.hashtag}{os.path.sep}{self.hashtag}_ms_cases.json'
        with open(path) as jf:
            data = json.load(jf)
        if len(data.keys()) > 1500:  
            print('only getting user feeds for 1.5k of our tweets. we must shorten our subset')
            return dict(take(1500,data.items()))
        else:
            return data

    def load_user_feeds(self):
        path = f'tweets/{self.hashtag}/100_feeds'
        jsons = [pos_json for pos_json in os.listdir(path) if pos_json.endswith('.json')]
        all_js = {}
        for file in jsons:
            with open(os.path.join(f'{path}/' + file)) as jf:
                all_js = { **all_js, **json.load(jf) }
        print(f'pulled data on {len(all_js)} users')
        return all_js


    @staticmethod
    def get_the_tweets(database):
        all_tweets = {}
        for key,value in database.items():
            #store tweets by tweet id
            all_tweets.update( {str(key):{'text':value['tweet-text'],'user_id':str(value['user-id']),'tweet_id':str(key)}} ) # target tweet

            inf = value['infector-info']
            k = list(inf.keys())[0]
            infector = inf[k]

            all_tweets.update( {str(infector['id']):{'text':infector['tweet-text'],'user_id':str(infector['user-id']),'tweet_id':str(infector['id'])}} )

            for i,informer in enumerate(value['informers-data']):
                if i <5:
                    all_tweets.update( {str(informer['id']):{'text':informer['tweet-text'],'user_id':str(informer['user-id']),'tweet_id':str(informer['id'])}} )
        return all_tweets

    @staticmethod
    def store_by_tweets(database):
        all_tweets = {}
        for key,value in database.items():
            if value in all_tweets:
                new = all_tweets[value].append(key)
                all_tweets[value] = new
            else:
                all_tweets[value] = [key]

        return all_tweets

    @staticmethod
    def get_users(database):
        users = {}
        for key,value in database.items():
            users.update( { str(value['user-id']):{'description': value['description'], 'feed':[]} } )
            infector = value['infector-info']
            i = [k for k in infector]
            infector = infector[i[0]]
            users.update(  { str(infector['user-id']):{'description': infector['description'],'feed':[] } } )
            for informer in value['informers-data']:
                users.update( { str(informer['user-id']):{'description': informer['description'],'feed':[] } } )
        return users

    def add_feeds(self,users):
        feeds = self.load_user_feeds()
        pulled_feeds = feeds.keys()
        users_got = users.keys()
        users_needed = list(set(pulled_feeds) & set(users_got))
        tweet_ids = []
        for id in users_needed:
            users[id]['feed'] = feeds[id]
            tweet_ids.extend( tw['id'] for tw in feeds[id]  )
        return users,tweet_ids

    @staticmethod
    def sort_by_tweet(all_tweets): 

        df = pd.DataFrame.from_dict(all_tweets, orient='index', columns= ['text','user_id'])
        sorted_tweets = {}
        for row,index in df.groupby('text').groups.items():
            key = tuple(index.values.tolist())
            sorted_tweets.update({key:row})

        new_df = pd.DataFrame.from_dict(sorted_tweets, orient='index', columns= ['text'])
        
        return new_df

    def tweet_cleaner(self,tw_list):
        remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
        rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
        hash = lambda x: re.sub(r'#', "", x)
        amp = lambda x: re.sub(r'&amp', "", x)


        tw_list['grammartext'] = tw_list.text.map(remove_rt).map(rt)
        tw_list['clean_text'] = tw_list.text.map(remove_rt).map(rt)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.HASHTAG)
        tw_list["grammartext"] = tw_list.grammartext.map(p.clean)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.NUMBER)
        tw_list["clean_text"] = tw_list.clean_text.map(p.clean).map(hash).map(amp)
        tw_list["clean_text"] = tw_list.clean_text.str.lower()
        return tw_list

    def remove_punct(self,text):
        text = "".join([char for char in text if char not in string.punctuation])
        text = re.sub('[0–9]+', '', text)
        return text


    def clean_text(self,text):
        text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
        text_rc = re.sub('[0-9]+', '', text_lc)
        tokens = re.split('\W+', text_rc)    # tokenization
        text = [self.ps.stem(word) for word in tokens if word not in self.stopword]  # remove stopwords and stemming
        return text



################################################################################################
################################################################################################
############################                 METRICS                ############################
################################################################################################
################################################################################################

    @staticmethod
    def get_sentiment(df,index,row):
        score = SentimentIntensityAnalyzer().polarity_scores(row.clean_text)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']
        comp = score['compound']
        if neg > pos:
            df.loc[index, 'sentiment'] = "negative"
        elif pos > neg:
            df.loc[index, 'sentiment'] = "positive"
        else:
            df.loc[index, 'sentiment'] = "neutral"
        df.loc[index, 'neg'] = neg
        df.loc[index, 'neu'] = neu
        df.loc[index, 'pos'] = pos
        df.loc[index, 'compound'] = comp

    def get_topic(self,df,index,row):
        # tokens = self.topic_tokenizer(row.clean_text,return_tensors='pt')
        output = self.topic_model(**row.topic_tokens.to(self.device))
        scores = output[0][0].detach().cpu().numpy()
        scores = expit(scores)
        pred = np.argmax(scores)

        df.loc[index,'topic'] = pred

        for i in range(19):
            label = str(self.topic_classes[i])
            df.loc[index, label] = scores[i]


    def get_hate(self,df,index,row):
        for i in range(3):
            df.loc[index, self.hate_labels[i]] = row.psy_output.probas[self.hate_labels[i]]


####################################################################################################
############################                 LOAD MODELS            ################################
####################################################################################################

    def load_topic_model(self):

        MODEL = f"cardiffnlp/tweet-topic-21-multi"
        tokenizer = AutoTokenizer.from_pretrained(MODEL)

        with torch.no_grad():
            self.topic_model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(self.device)

        self.topic_classes = self.topic_model.config.id2label

        print('now loading the topic tokens for all user feeds')
        feed_tokens = self.user_feeds.text.apply(lambda row: tokenizer(row, return_tensors='pt'))

        self.user_feeds['topic_tokens'] = feed_tokens
        print('---------\n---------\n')

    def load_psysentimento_model(self):

        # hateful
        analyzer = create_analyzer(task="hate_speech", lang="en")
        self.hate_labels = ['hateful', 'targeted', 'aggressive']

        print('loaded emotion model')
        feeds = self.user_feeds.text.to_list()
        print('loading psysentimento for user feeds')


        t1 = time.time()
        # the hate predictions are the first element, emotion predictions in the second
        fd_psy_pred = [ analyzer.predict(preprocess_tweet(txt))  for txt in feeds ]
        self.user_feeds['psy_output'] = fd_psy_pred
        t2 = time.time()
        print('predicted hate of feeds')
        print(f'in {(t2-t1)/3600} hours')
    
        


#######################################
# MAIN FUNCTION TO RUN THE ANALYSIS
########################################


    def tweet_analysis(self):

        informer_db = self.load_informer_data()
        self.get_device()

        print('loaded informer data')

        #####################
        # LOAD IN THE DATA!!!!!!!
        #####################

        # ALL THE TWEETS IN THE MULTI-SOURCE CASE - store by tweet id
        all_tweets = self.get_the_tweets(informer_db)
        self.df = pd.DataFrame.from_dict(all_tweets, orient='index')
        self.df.drop_duplicates()
        user_ids = self.df.user_id.copy().tolist()

        print('loaded in ms tweets')

        # ALL THE USER FEEDS!!!!! STORED BY THE USER ID !!!!!
        all_users = self.get_users(informer_db)
        all_users,tweet_ids_feeds = self.add_feeds(all_users)
        ## ALL USER FEEDS!! STORED BY THE TWEET ID!! WILL!!!
        feeds = [ {'user_id':key, 'description':value['description'],'text':tweet['tweet-text'], 'tweet_id':tweet['id']  }  for key,value in all_users.items() for tweet in value['feed']  ]
        self.user_feeds = pd.DataFrame(feeds)
        self.user_feeds['tweet_ids'] = tweet_ids_feeds
        self.user_feeds.set_index('tweet_ids', inplace=True)
        print(f'\n\n\n raw feed len {len(self.user_feeds)}')
        self.user_feeds.drop_duplicates()
        print(f'\n\n\n feed len after drop duplibats {len(self.user_feeds)}' )
        self.user_feeds = self.user_feeds.loc[self.user_feeds['user_id'].isin(user_ids)]
        print(f'\n\n\n feed len after not considering the feeds we didnt pull {len(self.user_feeds)}' )

        print('loaded in necessary data')

        ##################
        # DROPPING DUPLICATE TWEETS FROM BOTH DATAFRAMES....
        
        print(f'have {len(self.df)} tweets loaded in')
        self.df = self.df.copy()[~self.df.index.duplicated(keep='first')]
        print(f'now considering {len(self.df)} tweets')


        print(f'have {len(self.user_feeds)} tweets loaded in')
        self.user_feeds = self.user_feeds.copy()[~self.user_feeds.index.duplicated(keep='first')]
        print(f'now considering {len(self.user_feeds)} tweets')


        print('loading test data')

        #############################################################################################
        ########################################################################################
        ## SUBSET TESTING

        # self.df = self.df.iloc[0:10]
        # yusers = self.df['user_id'].tolist()
        # user_ids =yusers


        # self.user_feeds = self.user_feeds[self.user_feeds['user_id'].isin(yusers)]
        # print('loading test data')
        ## END
        ##########################################


        # load in ids
        feed_tweet_ids = list(self.user_feeds.index.values)
        tweet_ids = list(self.df.index.values)
        feed_user_ids = list(self.user_feeds['user_id'].astype(str))

        self.user_feeds[['polarity', 'subjectivity']] = self.user_feeds['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
        self.user_feeds = self.tweet_cleaner(self.user_feeds)

        
        ###############################
        ###############################
        ### LOADING IN NECESSARY MODELS

        
        if 'tweet_id' not in self.df:
            self.df['tweet_id'] = tweet_ids

        if 'tweet_id' not in self.user_feeds:
            self.df['tweet_id'] = feed_tweet_ids

        tweet_df = self.df.copy()
        del self.df

        loaders = [self.load_topic_model, self.load_psysentimento_model]

        [load() for load in loaders]

        user_feeds_df = self.user_feeds.copy()
        user_feeds_dum = self.user_feeds.copy()

        save_path_f = f'tweets/{self.hashtag}/{self.hashtag}_feeds_classified'
        # save_path_f = f'tweet_analysis/output/{self.hashtag}_feeds_classified2'

        ########################################################################

        with open( save_path_f + '.pkl', 'wb') as f:
            joblib.dump(self.user_feeds, f) # pkl

        del self.user_feeds

        return

        funcs = [self.get_sentiment, self.get_topic, self.get_hate]

        print('NOW SCORING EACH TWEET IN USER FEED')
        t1= time.time()
        # [ func(user_feeds_df,idx,tweet) for func in funcs for tweet, idx in user_feeds_dum.groupby('text').iterrows() ]




        [ func(user_feeds_df,index,row) for func in funcs for index, row in user_feeds_dum.iterrows() ]


        
        t2 = time.time()
        print(f'finshed scoring each tweet in users feed in {(t2-t1)/3600} hours')

        del user_feeds_dum  

        ########################################################################
        ### AVERAGE PER USER SCORE

        uid = list(set(tweet_df.user_id.tolist())) 

        fid = list(set(user_feeds_df.user_id.tolist())) 

        user_ids = user_ids = [ u for u in uid if u in fid ]


        tweet_df = tweet_df[ tweet_df['user_id'].notna()]
        user_feeds_df = user_feeds_df[ user_feeds_df['user_id'].notna()]


        tweet_df.reset_index(drop=False)
        tweet_df.set_index('user_id', inplace=True)

        user_feeds_df.reset_index(drop=False)
        user_feeds_df.set_index('user_id', inplace=True)


        topics = [self.topic_classes[i] for i in range(19)]
        del self.topic_classes
        del self.hate_labels

        print('now averaging users feeds')

        t1= time.time()
        for user in user_ids:
            self.get_user_means(str(user),tweet_df,user_feeds_df,topics)
            self.get_user_modes(str(user),tweet_df,user_feeds_df)
            self.get_counts(str(user),tweet_df,user_feeds_df,topics)
        t2= time.time()
        print(f'finshed averaging users feed in {(t2-t1)/3600} hours')


        tweet_df.reset_index(drop=False)
        tweet_df.set_index('tweet_id', inplace = True)
      
        save_path = f'tweets/{self.hashtag}/{self.hashtag}_USER_scores.csv'

        tweet_df.to_csv(save_path)


    def get_user_means(self,user,tweet_df,user_feeds_df,topics):
        user_info = user_feeds_df.loc[user]
        scores = ['neg','neu','pos','compound',
        'polarity', 'subjectivity',
        'hateful', 'targeted', 'aggressive']

        all_scores = scores + topics
        for score in all_scores:
            new = f'user_{score}_mean'
            tweet_df.loc[user,new] = user_info[score].mean()

    def get_user_modes(self,user,tweet_df,user_feeds_df):
        user_info = user_feeds_df.loc[user]
        scores = ['sentiment',
        'topic']
        for score in scores:
            new = f'user_{score}_mode'
            tweet_df.loc[user,new] = user_info[score].mode().tolist()[0]

    def get_counts(self,user,tweet_df,user_feeds_df, topics):
        user_sent = user_feeds_df.loc[user]['sentiment']
        user_topic = user_feeds_df.loc[user]['sentiment']
        for i in ['positive','negative','neutral']:
            new = f'user_{i}_count'
            tweet_df.loc[user,new] = len(user_sent[user_sent ==i ])
        for j in topics:
            tweet_df.loc[user,new] = len(user_topic[user_topic==j ])

## RUN

In [ ]:
# hashtags = ['avengers','blm','borisjohnson','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']

hashtags = ['monkeypox','blm','supercup','gaza']

for hashtag in hashtags:

    a = Analyzer(hashtag)

    a.tweet_analysis()

    print(f'----------------\nDONE \n{hashtag}hashtag  \n----------------')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB
loaded informer data
loaded in ms tweets
pulled data on 5241 users



 raw feed len 519552



 feed len after drop duplibats 519552



 feed len after not considering the feeds we didnt pull 519552
loaded in necessary data
have 6871 tweets loaded in
now considering 6871 tweets
have 519552 tweets loaded in
now considering 519552 tweets
loading test data
now loading the topic tokens for all user feeds
---------
---------



loading configuration file https://huggingface.co/finiteautomata/bertweet-base-emotion-analysis/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c246eed05359b1a49c45955b0265b488e35b0cbd2628e3ead7dd54c8815162ee.a2dff24b4e0a884c6d58a09968c5b68e7391e749eb698ad92541818d420fd01b
Model config RobertaConfig {
  "_name_or_path": "finiteautomata/bertweet-base-emotion-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "others",
    "1": "joy",
    "2": "sadness",
    "3": "anger",
    "4": "surprise",
    "5": "disgust",
    "6": "fear"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 3,
    "disgust": 5,
    "fear": 6,
    "joy": 1,
    "others": 0,
    "sad

loaded emotion model
loading psysentimento for user feeds
